In [1]:
from IPython.display import display, Javascript, HTML, clear_output
from ipywidgets import widgets, Output, interact, fixed
from ipywidgets.widgets import Dropdown, interactive_output, VBox, HBox
from urllib.parse import quote_plus
from utils.notebooks import get_date_slider_from_datetime

#display(Javascript('IPython.notebook.execute_cells_below()'))

/home/stankeaa/.pyenv/versions/3.7.0/envs/testvirtenv/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/stankeaa/.pyenv/versions/3.7.0/envs/testvirtenv/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
%%html
<style>
summary{
    display:list-item;
}
</style>

In [3]:
%%capture
## Some Extensions ##
%load_ext autoreload
%autoreload 2
%store -r the_page

if 'the_page' not in locals():
    import pickle
    print("Loading default data...")
    the_page = pickle.load(open("data/the_page.p",'rb'))

### <span style="color:green"> Modules Imported </span>

In [4]:
## Modules Imported ##

# Display
from IPython.display import display, Markdown as md, clear_output, HTML, Javascript
from datetime import datetime, date, timedelta
from ipywidgets import Layout
import urllib

# APIs
from wikiwho_wrapper import WikiWho
from external.wikipedia import WikipediaDV, WikipediaAPI

# Data Processing
import pandas as pd

# Visualization
import qgrid
from visualization.actions_listener import ActionsListener
from visualization.conflicts_listener import ConflictsListener
from visualization.talks_listener import TalksListener
from visualization.wordcloud_listener import WCListener, WCActionsListener
from visualization.views_listener import ViewsListener

from metrics.conflict import ConflictManager
from metrics.token import TokensManager
from utils.notebooks import get_next_notebook, get_previous_notebook

In [5]:
wikipedia_dv = WikipediaDV(WikipediaAPI(domain='en.wikipedia.org'))

# the method that listens to the click event
def on_button_clicked(b):
    global the_page, the_talk_page
    
    # use the out widget so the output is overwritten when two or more
    # searches are performed
    with out0:
        try:
            # query wikipedia
            search_result = wikipedia_dv.search_page(searchTerm.value)
            the_page = wikipedia_dv.get_page(search_result)
            the_talk_page = wikipedia_dv.get_page("Talk:"+search_result)
            %store the_page
            clear_output()
            display(md(f"The page that was found: **{the_page['title']}**"))
            display(md(f"Page id: **{the_page['page_id']}**"))

        except:
            clear_output()
            display(md(f'The page title *"{searchTerm.value}"* was not found'))

# by default display the last search
try:
    searchTerm = widgets.Text(the_page['title'], description='Page title:')
except:
    searchTerm = widgets.Text("The Camp of the Saints", description='Page title:')

display(searchTerm)

# create and display the button    
button = widgets.Button(description="Search")
display(button)

# the output widget is used to remove the output after the search field
out0 = Output()
display(out0)

# set the event
button.on_click(on_button_clicked)

# trigger the event with the default value
on_button_clicked(button)

Text(value='The Camp of the Saints', description='Page title:')

Button(description='Search', style=ButtonStyle())

Output()

If you have input another page title and still want to use these variables in the next 4 notebooks, please refresh!

In [ ]:
def run_below(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))

button = widgets.Button(description="Refresh All Variables", button_style='info', min_width=500)
button.on_click(run_below)
display(button)

---

# A. Change actions

**The dynamics of individual changes** to each word (and each special character) in the article are extracted by the [WikiWho API](https://www.wikiwho.net/en/api/v1.0.0-beta/). 

Below, you find several metrics for such change dynamics such as unsucessful/reverted changes and conflict. 

<br>
<details>    
    <summary style="cursor: pointer;font-weight:bold">More information about WikiWho's method</summary>


The [WikiWho API](https://www.wikiwho.net/en/api/v1.0.0-beta/) tracks the changes to every token (words or special characters) on a Wikipedia page with at least 95% accuracy, for live Wikipedia articles. It distinguishes every token in the document even when the string appears several times. E.g. "and" at the beginning of an article is a different token then "and" at the end of the article. See also [this figure](https://www.wikiwho.net/#technical_details).

That means that *not only edits* (== whole revisions) are counted, which can contain changes of many different tokens, but *every single action to every single token* is recorded. 
 <br>   
   The two most basic actions that can perfomed per token are **insertions** and **deletions**. A character change in a word, e.g. "dog" -> "dogs", is modeled as deletion of "dog" and the insertion of "dogs", two separate tokens.
 <br>   
    An **insertion** is also considered a **re-insertion** if the insertion has occured before; the only insertion of a token that is not a re-insertion is the first one (original addition of the token). Similarly, a **deletion** is also considered a **re-deletion** if the deletion has occured before.

Formally, the token history can represented by a time-ordered sequence of actions 
*(a<sub>0</sub>, ..., a<sub>n</sub>)*; note that *a<sub>0+2i</sub>* is always an insertion and 
*a<sub>1+2i</sub>* is always a deletion for i ∈ ℕ.
</details> 


---

*IMPORTANT:* For articles with a long revision history, please allow for some time to load, 
before interacting with the controls too often.

## A.1 When did certain change actions happen?

In the following graph you can
* select the *date range* and *granularity* (yearly, monthly) of the timeline (X-axis), and 
* plot any of the following *change action types* in the black, red, blue and green lines. 

<br>


<details>    
    <summary style="cursor: pointer;font-weight:bold">Explanation of the change action types</summary>

   
- **adds**: number of first-time insertions / additions
- **adds_surv_48h**: number of adds that survived at least 48 hours (after 48h, the risk of being removed is negligible on avg.)
- **adds_stopword_count**:  number of insertions that were only [stop words](https://github.com/gesiscss/wikiwho_demo_new/blob/master/data/stopword_list.txt)
<br>

- **dels**: number of deletions
- **dels_surv_48h**: number of deletions that were not resinserted in the next 48 hours 
- **dels_stopword_count**: number of deletions that were only [stop words](https://github.com/gesiscss/wikiwho_demo_new/blob/master/data/stopword_list.txt)
<br>

- **reins**: number of reinsertions
- **reins_surv_48h**: number of reinsertions that survived at least 48 hours
- **reins_stopword_count**: number of reinsertions that were only [stop words](https://github.com/gesiscss/wikiwho_demo_new/blob/master/data/stopword_list.txt)
</details>
<br>


<details>    
    <summary style="cursor: pointer;font-weight:bold">Example</summary>

For instance, if you see 10 "adds" in a month, but only 4 "adds_surv_48h", 10 completely new tokens/words have been added to the article, but only 4 of them stayed in the article for more than 2 days, which usually means the other 6 are gone for good. If "dels" are performed and don't survive, that means that these deletions have been undone, i.e., the deleted tokens have been put back. I.e., these are measurements of the longevity and stability of edit actions done to the article.
</details>

In [ ]:
# create and display the button
button2 = widgets.Button(description="Load Actions", layout=Layout(width='160px'))
display(button2)

def on_click_details(b):
    global agg_actions
    global editors
    global action_types
    with out4:
        clear_output()
        display(md(f"***Page: {the_page['title']}***"))
        
        wikiwho = WikiWho(lng='en')
        agg_actions = wikiwho.dv.edit_persistence(the_page.page_id)

        # Eliminate columns containing 'persistent'
        new_cols = agg_actions.columns[~agg_actions.columns.str.contains('persistent')]
        agg_actions = agg_actions[new_cols]

        # define total columns
        total_columns = ['total', 'total_surv_48h', 'total_stopword_count']

        # add columns with the total actions
        agg_actions = agg_actions.join(pd.DataFrame(
            agg_actions.loc[:,'adds':'adds_stopword_count'].values +\
            agg_actions.loc[:,'dels':'dels_stopword_count'].values +\
            agg_actions.loc[:,'reins':'reins_stopword_count'].values, 
            index=agg_actions.index, 
            columns=total_columns
        ))

        # the output widget is used to update the qgrid
        print("Downloading editor usernames (i.e. *editor* column)...")
        
        clear_output()
        # Grab user names from wikipedia and merge them to the editors_conflict dataframe
        wikipedia_dv = WikipediaDV(WikipediaAPI(domain='en.wikipedia.org'))
        editors = wikipedia_dv.get_editors(agg_actions['editor_id'].unique()).rename(columns = {
            'userid': 'editor_id'})

        # Merge the names of the editors to the aggregate actions dataframe
        display(md(f"***Page: {the_page['title']}***"))
        agg_actions = agg_actions.merge(editors[['editor_id', 'name']], on='editor_id')
        agg_actions.insert(3, 'editor', agg_actions['name'])
        agg_actions = agg_actions.drop(columns=['name'])
        agg_actions['editor'] = agg_actions['editor'].fillna("Unregistered")
                
        # Convert to datetime
        agg_actions['year_month'] = pd.to_datetime(agg_actions['year_month'])

        # Group the data by year month and page (drop the editor information)
        agg_actions.drop('editor_id', axis=1).groupby(['year_month','page_id']).sum().reset_index()

        # Listener
        listener = ActionsListener(agg_actions)
        action_types = (agg_actions.columns[4:13]).values.tolist()

        # Visualization
        datepicker_from = widgets.DatePicker(description='Date starts', value=agg_actions['year_month'].iloc[0], layout=Layout(width='25%'))
        datepicker_to = widgets.DatePicker(description='Date ends', value=agg_actions['year_month'].iloc[-1], layout=Layout(width='25%'))
#         out_action = interactive_output(listener.listen, {'_range1': datepicker_from, '_range2': datepicker_to, 'editor': fixed('All'),
#                                                          'granularity': Dropdown(options=['Yearly', 'Monthly'], value='Yearly'),
#                                                          'black': Dropdown(options=action_types, value='adds'),
#                                                          'red': Dropdown(options= ['None'] + action_types, value='dels'),
#                                                          'green': Dropdown(options= ['None'] + action_types, value='None'),
#                                                          'blue': Dropdown(options= ['None'] + action_types, value='None')})
        interact(listener.listen,
             _range1 = datepicker_from,
             _range2 = datepicker_to,
             editor=fixed('All'),
             granularity=Dropdown(options=['Yearly', 'Monthly'], value='Yearly'),
             black=Dropdown(options=action_types, value='adds'), 
             red=Dropdown(options= ['None'] + action_types, value='dels'),
             green=Dropdown(options= ['None'] + action_types, value='None'), 
             blue=Dropdown(options= ['None'] + action_types, value='None'))

    
out4 = Output()
display(out4)

# set the event
button2.on_click(on_click_details)

# trigger the event with the default value
on_click_details(button2)

<span style="color:MediumSeaGreen; font-weight:bold"> TRY YOURSELF! THIS IS WHAT WILL HAPPEN WHEN YOU CLICK 'Load Actions' BUTTON </span>

In [ ]:
### ----------------------------------------------------------------------------------- ###
### TRY YOURSELF! THIS IS WHAT WILL HAPPEN WHEN YOU CLICK 'Show Actions Details' BUTTON ###
### ----------------------------------------------------------------------------------- ###

## This is the page you used ##
print('The page that is being used:', the_page['title'])

## Create a WikiWho instance, which is an API; and use 'edit_persistence' method to get ##
## monthly editons for given page id; more details see:                                 ##
## https://github.com/gesiscss/wikiwho_wrapper/blob/master/wikiwho_wrapper/api.py       ##
## https://github.com/gesiscss/wikiwho_wrapper/blob/master/wikiwho_wrapper/wikiwho.py   ##
wikiwho_api = WikiWho(lng='en')
total_actions = wikiwho_api.dv.edit_persistence(the_page['page_id'])

## Eliminate columns containing 'persistent'
cols_no_persi = total_actions.columns[~total_actions.columns.str.contains('persistent')]
total_actions = total_actions[cols_no_persi]

## Convert some columns' names to what we could understand ##
total_columns_names = ['total', 'total_surv_48h', 'total_stopword_count']

## Add all actions together and merge them into one aggregation dataframe.
total_actions = total_actions.join(pd.DataFrame(
    total_actions.loc[:,'adds':'adds_stopword_count'].values +\
    total_actions.loc[:,'dels':'dels_stopword_count'].values +\
    total_actions.loc[:,'reins':'reins_stopword_count'].values, 
    index=total_actions.index, 
    columns=total_columns_names
))

## Only editors id? No, we still want to know their names. Use WikipediaAPI to achieve ##
## this. More details see:                                                             ##
## https://github.com/gesiscss/wikiwho_demo/blob/master/external/wikipedia.py          ##
wikipediadv_instance = WikipediaDV(WikipediaAPI(domain='en.wikipedia.org'))

# Unique editor's id for getting their names
unique_ids = total_actions['editor_id'].unique()

# Grab editors' names
grabbed_names = wikipediadv_instance.get_editors(unique_ids).rename(columns = {'userid': 'editor_id'})

## Merge the names of editors to the total actions dataframe ##

# Merge on editors ids
total_actions = total_actions.merge(grabbed_names[['editor_id', 'name']], on='editor_id')

# Adjust the column position and fill NaN by 'Unregistered'
total_actions.insert(3, 'editor', total_actions['name'])
total_actions = total_actions.drop(columns=['name'])
total_actions['editor'] = total_actions['editor'].fillna("Unregistered")

## Continue to use our data from A.1; first convert the date to the type datetime ##
total_actions['year_month'] = pd.to_datetime(total_actions['year_month'])

## visualization, core visual code lies in ActionsListener, then the interact function    ##
## make it interactive, mode details see:                                                 ##
## https://github.com/gesiscss/wikiwho_demo/blob/master/visualization/actions_listener.py ##

# Define a ActionsListener instance
actions_listener = ActionsListener(total_actions)

# Which types do those actions have?
actions_types = (total_actions.columns[4:13]).values.tolist()

# Visualization: try yourself by coding!
begin_date = date(2002, 3, 1)
end_date = date(2019, 4, 1)
editor_name = 'All'
frequency = 'Monthly' # 'Monthly', 'Yearly'
black_line = 'adds' # Choose one from 'actions_types'. Note that 'black_line doesn't have 'None' option but others have.
red_line = 'dels'
blue_line = 'None'
green_line = 'None'

actions_listener.listen(
    _range1 = begin_date,
    _range2 = end_date,
    editor = editor_name,
    granularity = frequency,
    black = black_line,
    red = red_line,
    blue = blue_line,
    green = green_line
)

# Store the variable total_actions for the next notebooks.
%store total_actions

## A.2 Who is responsible for these changes?

If you want to track revision history **until** the end of a month, select a year-month (row) in the below table and a link to Wikipedia's revision history will be provided underneath the table. (Notice: Although 'year_month' column displays the start of a month, it still contains all the actions implemented by this editor in that month.)

In [ ]:
# create and display the button
button2 = widgets.Button(description="Get Actions", layout=Layout(width='150px'))
display(button2)

# set the default max number of rows to 10 so the larger DataFrame we render don't take up to much space 
qgrid.set_grid_option('maxVisibleRows', 12)

def on_selection_change(change):
    with out21:
        clear_output()
        
        # Extract the time selected and convert it to string.
        time_selected = qgrid_obj.get_selected_df().reset_index()['year_month'].iloc[0]        
        date_for_search = pd.Series(time_selected).astype(str).iloc[0][0:8] + str(time_selected.daysinmonth)
        
        # Print URL.
        url = f"https://en.wikipedia.org/w/index.php?date-range-to={date_for_search}&tagfilter=&title={the_page['title'].replace(' ', '_')}&action=history"
        print(url)

def click_total_actions(b):
    global qgrid_obj
    with out2:
        clear_output()
        wikiwho = WikiWho(lng='en')

        # define total columns
        total_columns = ['total', 'total_surv_48h', 'total_stopword_count']
       
        # url
        url_a = wikiwho.dv.api.base_editor + f'/page/{the_page.page_id}'
        display(md(f"Actions data can be found in:"))
        print(url_a)
        
        qgrid_obj = qgrid.show_grid(agg_actions[['year_month', 'editor'] + action_types].set_index('year_month').sort_index(ascending=False),
                               grid_options={'forceFitColumns':False})
        display(qgrid_obj)
        
        qgrid_obj.observe(on_selection_change, names=['_selected_rows'])
        
        out21.clear_output()

out2 = Output()
display(out2)
out21 = Output()
display(out21)

# set the event
button2.on_click(click_total_actions)

# trigger the event with the default value
click_total_actions(button2)

<span style="color:MediumSeaGreen; font-weight:bold"> TRY YOURSELF! THIS IS WHAT WILL HAPPEN WHEN YOU CLICK 'Get Actions' BUTTON </span>

In [ ]:
### ----------------------------------------------------------------------------------- ###
### TRY YOURSELF! THIS IS WHAT WILL HAPPEN WHEN YOU CLICK 'Get Actions' BUTTON ###
### ----------------------------------------------------------------------------------- ###

## This is the page you used ##
print('The page that is being used:', the_page['title'])

## url: use the previous wikiwho_api instance##
url_actions = wikiwho_api.dv.api.base_editor + f'/page/{the_page.page_id}'
print("Actions data can be found in:", url_actions)

## Display the dataframe using interactive grid, you could learn more through the doc: ##
## https://qgrid.readthedocs.io/en/latest/                                             ##
display(qgrid.show_grid(total_actions[['year_month', 'editor'] + actions_types].set_index('year_month').sort_index(ascending=False),
                       grid_options={'forceFitColumns': False}))

## Select a month by coding and you can access all the revision history until this month through a link. ##
select_month = '2018-07-01'  # Any value in 'year_month' column.

# Change the first day to the end day of that month for Wikipedia filtering, 
# but both of them contain all the actions in that month.
day_in_that_month = pd.Timestamp(select_month).daysinmonth
select_month_end_day = select_month[0:8] + str(day_in_that_month)

# Print url.
url_revision_history = f"https://en.wikipedia.org/w/index.php?date-range-to={select_month_end_day}&tagfilter=&title={the_page['title'].replace(' ', '_')}&action=history"
print('Select one row with particular month then you can track all revision history until the end of this month on Wikipedia through a link.')
print(url_revision_history)

## A.3 Which words were changed?

The following WordCloud and table present the most 100 popular token strings that are either **added** or **undone (deleted/reinserted)**. The table also presents the detailed actions of each token.

<br>


<details>    
    <summary style="cursor: pointer;font-weight:bold">Explanation of the change action types</summary>

   
- **adds**: number of first-time insertions (additions)
- **adds_48h**: number of adds that survived at least 48 hours
- **adds_48h_ratio**: adds_48h / adds
<br>

- **dels**: number of deletions
- **dels_48h**: number of deletions that were not resinserted in the next 48 hours
- **dels_48h_ratio**: dels_48h / dels
<br>

- **reins**: number of reinsertions
- **reins_48h**: number of reinsertions that survived at least 48 hours
- **reins_48h_ratio**: reins_48h / reins
</details>

In case you are insterested in a particular token, e.g. "racist", just click the token (row) in the table and another table containing all revision history of this token within the selected date range will appear beneath the first table. Furthermore, if you want to investigate one action implemented on that token, then click that action (revision row) and a link to Wikipedia will be provided below for comparing different revisions.

In [ ]:
 # create and display the button
button3 = widgets.Button(description="Show WordCloud")
display(button3)

def on_click_wc_action(b):
    global all_actions
    with out12:
        clear_output()       
        wikiwho = WikiWho(lng='en')
        
        all_content = wikiwho.dv.all_content(the_page['page_id'])
        revisions = wikiwho.dv.rev_ids_of_article(the_page['page_id'])

        calculator = ConflictManager(all_content, revisions)
        calculator.calculate()
        clear_output()
        all_actions = calculator.all_actions
        
        # listener
        listener = WCActionsListener(sources = {
            'All actions': all_actions
        }, token_source=all_actions)

        # visualization
        datepicker_from = widgets.DatePicker(description='Date starts', value=date(2019, 3, 24), layout=Layout(width='25%'))
        datepicker_to = widgets.DatePicker(description='Date ends', value=date(2020, 3, 24), layout=Layout(width='25%'))
        
        _range1=datepicker_from
        _range2=datepicker_to
        source=Dropdown(options=list(listener.sources.keys()), value='All actions', description='Source (*)')
        action=Dropdown(options=['All', 'adds', 'adds_48h', 'dels', 'dels_48h', 'reins', 'reins_48h'], value='All', description='Action')

        out = interactive_output(listener.listen, {'_range1': _range1, '_range2':_range2, 'source':source, 'action':action})
        
        display(md(f"***Page: {the_page['title']}***"))
        #display(HBox([datepicker_from, datepicker_to]))
        display(VBox([datepicker_from, datepicker_to, action, out]))

out12 = Output()
display(out12)

# set the event
button3.on_click(on_click_wc_action)

# trigger the event with the default value
on_click_wc_action(button3)

---
# B. Conflicts

## B.1 When did conflicts happen?

Our measurement of ***conflict*** for single tokens is taken from [Flöck et al.](https://arxiv.org/abs/1703.08244):


* (1)  The main idea is to count how often a token  - after being created (added) the first time -  was being deleted, re-inserted, re-deleted, re-inserted, and so on; which would often happen in case two editors disagree on the token's justification to be in the text. 
* (2) Only the **re-**deletions and **re-**insertions are counted, since up to the first delete it could be a simple correction that didn't trigger a reponse - this wouldn't indicate conflict. 
* (3) The **"re-"** actions are only counted if they alternate between different editors and don't come from the same editor twice or more in a row - as the latter would simply indicate self-corrections. 
* (4) In a last step, each re-insertion/re-deletion interaction gets a higher weight the faster it occurs (see [Flöck et al.](https://arxiv.org/abs/1703.08244) for the exact formula).

The total conflict of a page is the sum of all the conflict scores of all actions with 
conflict (or conflict actions).

<mark> *Question: What is the exact definition of conflict scores of actions?* </mark>

This total conflict can be normalized if the sum is divided by the number of 
actions that could potentially be counted as conflict (elegible actions, i.e **"re-"** actions that have occurred at 
least twice).


In the following graph you can select the *date range* and *granularity* (yearly, monthly) 
of the timeline (X-axis), and plot any of the following counts in the black and red lines:
<details>    
    <summary style="cursor: pointer;font-weight:bold">Options description</summary>
    
- **Total**: total number of actions (insertions, and deletions)
- **Total_surv_48h**: total number of actions that survived at least 48 hours
- **Total_persistent**: total number of actions that survived until, at least, the end of the month
- **Total_stopword_count**: total number of actions that were performed in stop words
- **Total Elegible Actions**: the total number of elegible actions
- **Number of Conflicts**: the total number of conflicts
- **Number of Revisions**: the total number of revisions/edits
- **Conflict Score**: the sum of conflict scores of all actions divided by the number of elegible actions
- **Absolute Conflict Score**: the sum of conflict scores of all actions (without division)
- **Conflict Ratio**: the count of all conflicts divided by the number of elegible actions
</details>

In [ ]:
# create and display the button
button2 = widgets.Button(description="Show Page Conflict", layout=Layout(width='160px'))
display(button2)

def on_click_conflict(b):
    global datepicker_from
    with out6:
        clear_output()
        display(md(f"***Page: {the_page['title']}***"))
        
        # Visualization
        listener = ConflictsListener(agg_actions)

        metrics = ['Conflict Score', 'Total Elegible Actions', 'Absolute Conflict Score']
        conflict_score = agg_actions.conflict.sum() / agg_actions.elegibles.sum()
        display(md(f'**Page conflict score: {conflict_score}**'))
        
        # Visualization
        datepicker_from = widgets.DatePicker(description='Date starts', value=agg_actions['year_month'].iloc[0], layout=Layout(width='25%'))
        datepicker_to = widgets.DatePicker(description='Date ends', value=agg_actions['year_month'].iloc[-1], layout=Layout(width='25%'))
        if (conflict_score != 0):
            interact(listener.listen,
                     _range1 = datepicker_from,
                     _range2 = datepicker_to,
                     granularity=Dropdown(options=['Yearly', 'Monthly'], value='Monthly'),
                     black=Dropdown(options=metrics, value='Conflict Score'),
                     red=Dropdown(options= ['None'] + metrics, value='None'))
        #display(Javascript('Jupyter.notebook.execute_cells([24])'))
    

out6 = Output()
display(out6)

# set the event
button2.on_click(on_click_conflict)

# trigger the event with the default value
on_click_conflict(button2)

<span style="color:MediumSeaGreen; font-weight:bold"> TRY YOURSELF! THIS IS WHAT WILL HAPPEN WHEN YOU CLICK 'Show Page Conflict' BUTTON </span>

In [ ]:
### ----------------------------------------------------------------------------------- ###
### TRY YOURSELF! THIS IS WHAT WILL HAPPEN WHEN YOU CLICK 'Show Page Conflict' BUTTON   ###
### ----------------------------------------------------------------------------------- ###

## This is the page you used ##
print('The page that is being used:', the_page['title'])

## Continue to use our data from A.2; calculate the Conflict Score definded as above ##
conflict_sc = total_actions.conflict.sum() / total_actions.elegibles.sum()
print('Page conflict score is:', conflict_sc)

## visualization, core visual code lies in ConflictsListener, then the interact function    ##
## make it interactive, mode details see:                                                   ##
## https://github.com/gesiscss/wikiwho_demo/blob/master/visualization/conflicts_listener.py ##

# First check whether the page conflict score is zero
if conflict_sc != 0:
    # Define a ConflictsListener instance
    conflicts_listener = ConflictsListener(total_actions)

    # Set parameters
    begin_date = total_actions['year_month'].iloc[0]
    end_date = total_actions['year_month'].iloc[-1]
    frequency = 'Yearly' # 'Monthly'
    
    # The metrics we need:
    # ['Total', 'Total_surv_48h', 'Total_persistent', 'Total_stopword_count',
    #  'Total Elegible Actions', 'Number of Conflicts', 'Number of Revisions',
    #  'Conflict Score', 'Absolute Conflict Score', 'Conflict Ratio', ('None')]
    # Note: only 'red_line' has 'None' option.
    black_line = 'Conflict Score'
    red_line = 'None'
    
    conflicts_listener.listen(
        _range1 = begin_date,
        _range2 = end_date,
        granularity = frequency,
        black = black_line,
        red = red_line
    )

---

## B.2 Editor Conflict Score

We can also calculate the conflict score for each individual editor. The
table below presents the conflict score and other related  metrics per editor (*editor_id* and *editor*
column):

<details>    
    <summary style="cursor: pointer;font-weight:bold">Columns description</summary>

- **conflicts**: the total number of conflicts
- **elegibles**: the total number of elegible actions performed by the editor
- **conflict**: the sum of conflict scores of all actions divided by the number of elegible actions
</details>

In [ ]:
# create and display the button
button2 = widgets.Button(description="Show Editor's Conflicts", layout=Layout(width='160px'))
display(button2)

def on_click_editor(b):
    global editors_conflicts
    with out7:
        clear_output()
        display(md(f"***Page: {the_page['title']}***"))
                   
        editors_conflicts = agg_actions.groupby(pd.Grouper(
                    key='editor_id')).agg({'conflicts': 'sum', 'elegibles': 'sum', 'conflict': 'sum'}).reset_index()
        editors_conflicts['conflict'] = (editors_conflicts['conflict']/editors_conflicts['elegibles'])
        if len(editors_conflicts) > 0:
            editors_conflicts = editors[['editor_id', 'name']].merge(editors_conflicts.dropna(), 
                                                        right_index=True, on='editor_id').set_index('editor_id')
            editor_conflicts_noid = editors_conflicts.dropna().set_index('name')[['elegibles', 'conflicts', 'conflict']]
            qg_obj = qgrid.show_grid(editor_conflicts_noid)
            display(qg_obj)
            #display(Javascript('Jupyter.notebook.execute_cells([28])'))
                   
        else:
            display(md(f'**There is no Conflict Scores**')) 
            editors_conflicts = None
            
out7 = Output()
display(out7)

# set the event
button2.on_click(on_click_editor)

# trigger the event with the default value
on_click_editor(button2)

<span style="color:MediumSeaGreen; font-weight:bold"> TRY YOURSELF! THIS IS WHAT WILL HAPPEN WHEN YOU CLICK 'Show Editor's Conflicts' BUTTON </span>

In [ ]:
### -------------------------------------------------------------------------------------- ###
### TRY YOURSELF! THIS IS WHAT WILL HAPPEN WHEN YOU CLICK 'Show Editor's Conflicts' BUTTON ###
### -------------------------------------------------------------------------------------- ###

## This is the page you used ##
print('The page that is being used:', the_page['title'])

## Continue to use our data from A.2; sum 'conflicts', 'elegibles' and 'conflict' (absolute) up by editors ##
conflicts_by_editors = total_actions.groupby(pd.Grouper(
                            key='editor_id')).agg({'conflicts': 'sum', 'elegibles': 'sum', 'conflict': 'sum'}).reset_index()

## Convert 'conflict' (absolute) to conflict ##
conflicts_by_editors['conflict'] = (conflicts_by_editors['conflict'] / conflicts_by_editors['elegibles'])

## If there are conflicts, then display it using qgrid ##
if len(conflicts_by_editors) > 0:
    conflicts_by_editors = grabbed_names[['editor_id', 'name']].merge(conflicts_by_editors.dropna(), 
                                                    right_index=True, on='editor_id').set_index('editor_id')
    conflicts_by_editors_noid = conflicts_by_editors.dropna().set_index('name')[['elegibles', 'conflicts', 'conflict']]
    qg_obj = qgrid.show_grid(conflicts_by_editors_noid)
    display(qg_obj)
else:
    display(md(f'**There is no Conflict Scores**')) 
    conflicts_by_editors = None

## Store the variable for the next notebook. ##
%store conflicts_by_editors

## B.3 Conflict score of each singular action

In the following table, all the actions that are in conflict are presented, and a conflict score
is presented per action. The importance of the conflict can be meassure by considering the seconds t that 
have passed since the last action on the same token has occured (`time_diff_secs` column). A score to 
meassure conflict is calculated based on t with the following formula: 1 / log<sub>3600</sub>(t+2). 
Thus, *undo* actions are weighted higher than the original time in seconds when the *t* is less than an hour.
For details, please refer to [Flöck et al, 2017](https://arxiv.org/abs/1703.08244).

<details>    
    <summary style="cursor: pointer;font-weight:bold">Columns description</summary>

- **token**: the string of the token that is being tracked
- **token_id**: the id of the token that is being tracked
- **rev_id**: the revision id in which the action (insertion or deletion) happen
- **editor_id**: the id of the editor that inserted the token (if starts with **0|**, it means that
the editor is not registered, and the ip is displayed instead
- **time_diff_secs**: seconds that have passed since the last action on the same token has occured
- **conflict**: a score to meassure conflict that is calculated based on the `time_diff_secs` 
with the following formula: *1 / log<sub>3600</sub>(time_diff_secs + 2)*. For details, please refer to 
[Flöck et al, 2017](https://arxiv.org/abs/1703.08244)

<mark> *Question: Should explain -1 in the rev_id column?* </mark>
</details>

In [29]:
# create and display the button
button2 = widgets.Button(description="Show Conflict Score of Each Action", layout=Layout(width='250px'))
display(button2)

def on_click_actions(b):
    global calculator
    with out9:
        clear_output()
        # create the api
        wikiwho = WikiWho(lng='en')

        # Get the content and revisions from the wikiwho api
        display(md("Downloading all_content from the WikiWhoApi..."))
        all_content = wikiwho.dv.all_content(the_page['page_id'])

        display(md("Downloading revisions from the WikiWhoApi..."))
        revisions = wikiwho.dv.rev_ids_of_article(the_page['page_id'])

        clear_output()

        # call the calculator
        calculator = ConflictManager(all_content, revisions)
        calculator.calculate()
        clear_output()

        display(md(f"***Page: {the_page['title']}***"))
        # display the tokens, the difference in seconds and its corresponding conflict score
        conflicts = calculator.conflicts.copy()
        conflicts['time_diff_secs'] = conflicts['time_diff'].dt.total_seconds()

        if len(conflicts) > 0:
            conflicts_for_grid = conflicts[[
                'action', 'token', 'token_id', 'rev_id', 
                'editor', 'time_diff_secs', 'conflict']].rename(columns={
                'editor': 'editor_id'}).sort_values('conflict', ascending=False)
            conflicts_for_grid['token_id'] = conflicts_for_grid['token_id'].astype(str)
            conflicts_for_grid['rev_id'] = conflicts_for_grid['rev_id'].astype(str)
            conflicts_for_grid.set_index('token_id', inplace=True)
            display(qgrid.show_grid(conflicts_for_grid))
        else:
            display(md(f'**There are no conflicting tokens in this page.**'))
            display(HTML(f'<a href="{get_previous_notebook()}" target="_blank">Go back to the previous workbook</a>'))
                   
        #display(Javascript('Jupyter.notebook.execute_cells([34])'))
                   
out9 = Output()
display(out9)

# set the event
button2.on_click(on_click_actions)

# trigger the event with the default value
on_click_actions(button2)

Button(description='Show Conflict Score of Each Action', layout=Layout(width='250px'), style=ButtonStyle())

Output()

<span style="color:MediumSeaGreen; font-weight:bold"> TRY YOURSELF! THIS IS WHAT WILL HAPPEN WHEN YOU CLICK 'Show Conflict Score of Each Action' BUTTON </span>

In [28]:
### ----------------------------------------------------------------------------------- ###
### TRY YOURSELF! THIS IS WHAT WILL HAPPEN WHEN YOU CLICK 'Conflict By Action' BUTTON   ###
### ----------------------------------------------------------------------------------- ###

## Get the content and revisions from the wikiwho api ##
page_content = wikiwho_api.dv.all_content(the_page['page_id'])
page_revisions = wikiwho_api.dv.rev_ids_of_article(the_page['page_id'])

## Call the calculator, you could learn more about it through               ##
## https://github.com/gesiscss/wikiwho_demo/blob/master/metrics/conflict.py ##
conflict_calculator = ConflictManager(page_content, page_revisions)
conflict_calculator.calculate()
clear_output()

## This is the page you used ##
print('The page that is being used:', the_page['title'])

## Display the tokens, the difference in seconds and its corresponding conflict score                 ##
## Pandas.Series.dt: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html ##
token_conflicts = conflict_calculator.conflicts.copy()
token_conflicts['time_diff_secs'] = token_conflicts['time_diff'].dt.total_seconds()

## Display using qgrid ##
if len(token_conflicts) > 0:
    tokens_grid_show = token_conflicts[[
        'action', 'token', 'token_id', 'rev_id', 
        'editor', 'time_diff_secs', 'conflict']].rename(columns={
        'editor': 'editor_id'}).sort_values('conflict', ascending=False)
    tokens_grid_show['token_id'] = tokens_grid_show['token_id'].astype(str)
    tokens_grid_show['rev_id'] = tokens_grid_show['rev_id'].astype(str)
    tokens_grid_show.set_index('token_id', inplace=True)
    display(qgrid.show_grid(tokens_grid_show))
else:
    display(md(f'**There are no conflicting tokens in this page.**'))
    display(HTML(f'<a href="{get_previous_notebook()}" target="_blank">Go back to the previous workbook</a>'))
    
# Store the variable for the next notebook.
%store conflict_calculator

NameError: name 'wikiwho_api' is not defined

---

## B.4 Most frequent conflicting token strings

The WordCloud displays the most common conflicting token strings, i.e. words (token strings) 
with the most actions that have conflict. The size of the token string in the WordCloud indicates frequency 
of actions.
In the controls you can select the *date range*, the type of *action* (insertion or deletion), and the 
*source*.

<details>    
    <summary style="cursor: pointer;font-weight:bold">Source description</summary>
    
-   **Only Conflicts**: use only the actions that are in conflict.
-   **Elegible Actions**: use only the actions that can potentially enter into conflict, i.e. actions 
that have occurred at least twice, e.g. the token x has been inserted twice (which necessarily implies 
it was remove once), the token x has been deleted twice (which necessarily implies it was inserted twice) 
-   **All Actions**: use all tokens regardles conflict
</details>

In [ ]:
# create and display the button
button2 = widgets.Button(description="Show WordCloud")
display(button2)

def on_click_wc(b):
    with out11:
        clear_output()
        display(md(f"***Page: {the_page['title']}***"))
        # listener
        listener = WCListener(sources = {
            'All actions': calculator.all_actions,
            'Elegible Actions': calculator.elegible_actions,
            'Only Conflicts': calculator.conflicts
        })

        # visualization
        datepicker_from = widgets.DatePicker(description='Date starts', value=date(2019, 3, 24), layout=Layout(width='25%'))
        datepicker_to = widgets.DatePicker(description='Date ends', value=date(2020, 3, 24), layout=Layout(width='25%'))        
        
        source=Dropdown(options=list(listener.sources.keys()), value='Only Conflicts', description='Source (*)')
        action=Dropdown(options=['Both', 'Just Insertions', 'Just Deletions'], value='Both', description='Action')
        editor=fixed('All')

        out = interactive_output(listener.listen, {
                 '_range1': datepicker_from,
                 '_range2': datepicker_to,
                 'source': source,
                 'action': action,
                 'editor': editor})

        display(VBox([datepicker_from, datepicker_to, action, source, out]))
        #display(Javascript('Jupyter.notebook.execute_cells([40])'))

out11 = Output()
display(out11)

# set the event
button2.on_click(on_click_wc)

# trigger the event with the default value
on_click_wc(button2)

<span style="color:MediumSeaGreen; font-weight:bold"> TRY YOURSELF! THIS IS WHAT WILL HAPPEN WHEN YOU CLICK 'Show WordCloud' BUTTON </span>

In [ ]:
### ----------------------------------------------------------------------------------- ###
### TRY YOURSELF! THIS IS WHAT WILL HAPPEN WHEN YOU CLICK 'Show WordCloud' BUTTON       ###
### ----------------------------------------------------------------------------------- ###

## This is the page you used ##
print('The page that is being used:', the_page['title'])

## WordCloud, core visual code lies in WCListener, then the interact function               ##
## make it interactive, mode details see:                                                   ##
## https://github.com/gesiscss/wikiwho_demo/blob/master/visualization/wordcloud_listener.py ##

# Create a WCListener instance, continue to use our data from B.1;
wclistener = WCListener(sources = {
    'All actions': conflict_calculator.all_actions,
    'Elegible Actions': conflict_calculator.elegible_actions,
    'Only Conflicts': conflict_calculator.conflicts
})

# visualization: you could also perform it by coding!
begin_date = date(2005, 3, 1)
end_date = date(2015, 7, 4)
actions_source='Only Conflicts' # 'Elegible Actions', 'All actions'
action_type='Both' # 'Just Insertions', 'Just Deletions'
editor='All'

print('Time range from', begin_date.strftime("%Y-%m-%d"), 'to', end_date.strftime("%Y-%m-%d"))

wclistener.listen(
     _range1=begin_date,
     _range2=end_date,
     source=actions_source,
     action=action_type,
     editor=editor)

---
# C. Talk

## C.1 How are comments in talk section spread over time?

Description of the Talk section

In [6]:
button_t = widgets.Button(description="Load Talks", layout=Layout(width='160px'))
display(button_t)

def on_click_details(b):
    global talk_content 
    with out_t:
        clear_output()
        print('The page that is being used:', the_talk_page['title'])

        #Initiate a Wikipedia instance
        wikipediadv_instance = WikipediaDV(WikipediaAPI(domain='en.wikipedia.org'))
        
        #Get content of revisions on talk page
        talk_content = wikipediadv_instance.get_talk_content(the_talk_page["page_id"])
        
        #convert to datetime format
        talk_content['year_month'] = pd.to_datetime(talk_content['timestamp'])

        # Define a TalksListener instance
        talks_listener = TalksListener(talk_content)

        # Visualization
        datepicker_from = widgets.DatePicker(description='Date starts', value=talk_content['year_month'].iloc[-1], layout=Layout(width='25%'))
        datepicker_to = widgets.DatePicker(description='Date ends', value=talk_content['year_month'].iloc[0], layout=Layout(width='25%'))

        interact(talks_listener.listen,
            begin = datepicker_from,
            end = datepicker_to,
            granularity = Dropdown(options=['Monthly', 'Yearly'], value='Monthly')
        )

out_t = Output()
display(out_t)

# set the event
button_t.on_click(on_click_details)

# trigger the event with the default value
on_click_details(button_t)


Button(description='Load Talks', layout=Layout(width='160px'), style=ButtonStyle())

Output()


## C.2 What are the most comments about?

In [34]:
import re

#extracting the topic,usually the subheaders in the Talk page
def find_topic(comment):
    if re.search('\/\*(.+?)\*\/', comment):
        return re.search('\/\*(.+?)\*\/', comment).group(1)
    else:
        return comment

talk_content['topics'] = talk_content.comment.apply(lambda x: find_topic(x))
topic_df = talk_content.groupby(by="topics").count().sort_values('user', ascending=False)

topic_df = topic_df.drop(["timestamp", "comment", "anon", "year_month"], axis=1).rename(columns = {
            'user': 'counts'})
display(qgrid.show_grid(topic_df))

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [ ]:
%store agg_actions
%store calculator
%store editors_conflicts

clear_output()
if len(editors_conflicts) > 0:
    display(HTML(f'<a href="{get_next_notebook()}" target="_blank">Go to next workbook</a>'))
else:
    display(HTML(f'<a href="{get_previous_notebook()}" target="_blank">Go back to the previous workbook</a>'))

scroll_to_top = """
<script>
document.getElementById('notebook').scrollIntoView();
</script>
"""
display(HTML(scroll_to_top))